In [1]:
import pandas as pd

In [2]:
# importing gl df

gl = pd.read_csv(r'C:\Users\CharlesYi\Jupyter Notebook\Alteryx Challenges\Challenge 457_Profit and Loss Statement\GL.csv')

gl.head()

,EntryNo,Date,Territory_key,Account_key,Details,Amount
0,1.1,2024-01-01,1,230,Cost of Sales,-884
1,2.1,2024-01-01,1,230,Cost of Sales,-1120
2,3.1,2024-01-01,1,280,Credit Expenses,-2394
3,4.1,2024-01-01,1,210,Credit Sales,2948
4,5.1,2024-01-01,1,210,Cash Sales,3734


In [3]:
# importing chart of accounts df

chart = pd.read_csv(r'C:\Users\CharlesYi\Jupyter Notebook\Alteryx Challenges\Challenge 457_Profit and Loss Statement\Chart of Accounts.csv')

chart.head()

,Account_key,Report,Class,Account
0,210,Profit and Loss,Sales,Sales
1,220,Profit and Loss,Sales,Sales Return
2,230,Profit and Loss,Cost of Sales,Cost of Sales
3,240,Profit and Loss,Operating Expenses,Staff Costs
4,250,Profit and Loss,Operating Expenses,Bad Debt Expense


In [4]:
# joining the two tables on account key

joined = pd.merge(gl, chart, on = 'Account_key')[['Class', 'Amount', 'Date']]

joined.head()

,Class,Amount,Date
0,Cost of Sales,-884,2024-01-01
1,Cost of Sales,-1120,2024-01-01
2,Cost of Sales,-4371,2024-01-01
3,Cost of Sales,-395,2024-01-05
4,Cost of Sales,-1165,2024-01-13


In [5]:
# checking dtype

joined.dtypes

Class     object
Amount     int64
Date      object
dtype: object

In [6]:
# changing date field to date dtype

joined['Date'] = pd.to_datetime(joined['Date'])

# checking dtype

joined['Date'].dtype

dtype('<M8[ns]')

In [7]:
# extracting month number from date

joined['Month_Num'] = joined['Date'].dt.month

joined.head()

,Class,Amount,Date,Month_Num
0,Cost of Sales,-884,2024-01-01,1
1,Cost of Sales,-1120,2024-01-01,1
2,Cost of Sales,-4371,2024-01-01,1
3,Cost of Sales,-395,2024-01-05,1
4,Cost of Sales,-1165,2024-01-13,1


In [8]:
# extracting month name from date

joined['Month'] = joined['Date'].dt.strftime('%B')

joined.head()

,Class,Amount,Date,Month_Num,Month
0,Cost of Sales,-884,2024-01-01,1,January
1,Cost of Sales,-1120,2024-01-01,1,January
2,Cost of Sales,-4371,2024-01-01,1,January
3,Cost of Sales,-395,2024-01-05,1,January
4,Cost of Sales,-1165,2024-01-13,1,January


In [9]:
# aggregating amount by class and month

agg = joined.groupby(['Class', 'Month', 'Month_Num'])['Amount'].sum().reset_index()

agg.head()

,Class,Month,Month_Num,Amount
0,Cost of Sales,April,4,-362082
1,Cost of Sales,August,8,-539100
2,Cost of Sales,December,12,-588184
3,Cost of Sales,February,2,-212342
4,Cost of Sales,January,1,-333820


In [10]:
# crosstabbing month and amount on class

crosstab = pd.crosstab(agg['Class'], agg['Month_Num'], agg['Amount'], aggfunc = 'sum')

crosstab.head()

Month_Num,1,2,3,4,5,6,7,8,9,10,11,12
Class,,,,,,,,,,,,
Cost of Sales,-333820.0,-212342.0,-286774.0,-362082.0,-345751.0,-339505.0,-558773.0,-539100.0,-522478.0,-591033.0,-735648.0,-588184.0
Depreciation & Amortization,-137825.0,-137825.0,-137825.0,-137825.0,-137825.0,-137825.0,-137825.0,-137825.0,-137825.0,-137825.0,-137825.0,-137825.0
Dividend,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67698.0,NaN
Exchange Loss/Gain,988.0,317.0,1699.0,1163.0,1368.0,487.0,1072.0,-348.0,420.0,1758.0,1555.0,456.0
Gain/Loss on Sales of Asset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16286.0


In [12]:
# creating dictionary to assoicate month number with month name

unique_months = agg[['Month', 'Month_Num']].drop_duplicates()

month_dict = dict(zip(unique_months['Month_Num'], unique_months['Month']))

month_dict

{4: 'April',
 8: 'August',
 12: 'December',
 2: 'February',
 1: 'January',
 7: 'July',
 6: 'June',
 3: 'March',
 5: 'May',
 11: 'November',
 10: 'October',
 9: 'September'}

In [13]:
# replacing column names in crosstab based on month_dict

crosstab.rename(columns = month_dict)

Month_Num,January,February,March,April,May,June,July,August,September,October,November,December
Class,,,,,,,,,,,,
Cost of Sales,-333820.0,-212342.0,-286774.0,-362082.0,-345751.0,-339505.0,-558773.0,-539100.0,-522478.0,-591033.0,-735648.0,-588184.0
Depreciation & Amortization,-137825.0,-137825.0,-137825.0,-137825.0,-137825.0,-137825.0,-137825.0,-137825.0,-137825.0,-137825.0,-137825.0,-137825.0
Dividend,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67698.0,NaN
Exchange Loss/Gain,988.0,317.0,1699.0,1163.0,1368.0,487.0,1072.0,-348.0,420.0,1758.0,1555.0,456.0
Gain/Loss on Sales of Asset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16286.0
Interest,-657.0,3180.0,1794.0,637.0,-183.0,-424.0,-1203.0,-1889.0,-1265.0,-1161.0,-798.0,110.0
Operating Expenses,-466506.0,-445218.0,-523234.0,-481339.0,-569877.0,-512179.0,-618477.0,-576509.0,-563058.0,-554415.0,-590561.0,-399224.0
Sales,1046468.0,651190.0,894417.0,1103670.0,1120884.0,1067888.0,1777538.0,1713521.0,1649697.0,1831216.0,2384673.0,1867480.0
Tax,-9846.0,33171.0,22898.0,-19486.0,-5809.0,-8001.0,-86142.0,-79868.0,-68297.0,-96507.0,-167649.0,-129231.0
